In [1]:
from pyspark.sql import SparkSession, functions
from pyspark import SparkContext, SparkConf, sql
import os
from dotenv import load_dotenv
import boto3
from pyspark.sql import SparkSession
import requests
import logging

In [2]:
conf = SparkConf().setAppName('spark_app').setMaster('local').setAll( [('spark.driver.memory', '16G'),
            ('spark.rpc.message.maxSize', 2000),
            ('spark.executor.memory', '4G'),
            ('spark.executor.cores', '4'),
            ('spark.kryoserializer.buffer.max','2000m'),
            ('spark.driver.maxResultSize', '12G'),
            ("spark.sql.execution.arrow.enabled", "true"),
            ("spark.dynamicAllocation.minExecutors", 4),
            ("spark.dynamicAllocation.maxExecutors", 20),
            ("spark.sql.caseSensitive", "true")])

In [3]:
spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark.sparkContext.setLogLevel('WARN')

In [4]:
spark.conf.set('spark.sql.parquet.mergeSchema', 'true')

In [5]:
spark.stop()

In [8]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("PySpark+MinIO") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000/") \
    .config("spark.hadoop.fs.s3a.access.key", "minio") \
    .config("spark.hadoop.fs.s3a.secret.key", "minio123") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.4.1,software.amazon.awssdk:bundle:2.29.6,software.amazon.awssdk:core:2.29.6") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .getOrCreate()

BUCKET = 'new'
df = spark.read.csv(f"s3a://{BUCKET}/data.csv")

Py4JJavaError: An error occurred while calling o65.csv.
: java.lang.NoClassDefFoundError: org/apache/hadoop/fs/BulkDelete
	at java.base/java.lang.Class.forName0(Native Method)
	at java.base/java.lang.Class.forName(Class.java:398)
	at org.apache.hadoop.conf.Configuration.getClassByNameOrNull(Configuration.java:2625)
	at org.apache.hadoop.conf.Configuration.getClassByName(Configuration.java:2590)
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2686)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.streamin

In [1]:
from io import BytesIO
from pathlib import Path
from typing import Optional, Union

import boto3
from botocore.client import Config
from botocore.exceptions import ClientError
from botocore.response import StreamingBody


class S3BucketService:
    def __init__(
        self,
        bucket_name: str,
        endpoint: str,
        access_key: str,
        secret_key: str,
    ) -> None:
        self.bucket_name = bucket_name
        self.endpoint = endpoint
        self.access_key = access_key
        self.secret_key = secret_key

    def create_s3_client(self) -> boto3.client:
        client = boto3.client(
            "s3",
            endpoint_url=self.endpoint,
            aws_access_key_id=self.access_key,
            aws_secret_access_key=self.secret_key,
            config=Config(signature_version="s3v4"),
        )
        return client

    def upload_file_object(
        self,
        prefix: str,
        source_file_name: str,
        content: Union[str, bytes],
    ) -> None:
        client = self.create_s3_client()
        destination_path = str(Path(prefix, source_file_name))

        if isinstance(content, bytes):
            buffer = BytesIO(content)
        else:
            buffer = BytesIO(content.encode("utf-8"))
        client.upload_fileobj(buffer, self.bucket_name, destination_path)

    def list_objects(self, prefix: str) -> list[str]:
        client = self.create_s3_client()

        response = client.list_objects_v2(Bucket=self.bucket_name, Prefix=prefix)
        storage_content: list[str] = []

        try:
            contents = response["Contents"]
        except KeyError:
            return storage_content

        for item in contents:
            storage_content.append(item["Key"])

        return storage_content

    def delete_file_object(self, prefix: str, source_file_name: str) -> None:
        client = self.create_s3_client()
        path_to_file = str(Path(prefix, source_file_name))
        client.delete_object(Bucket=self.bucket_name, Key=path_to_file)

In [3]:
import configparser


config = configparser.ConfigParser()
config.read('default.ini')

s3 = s3_bucket_service_factory(config)
s3.upload_file_object("test", "test.txt", "test")

NameError: name 's3_bucket_service_factory' is not defined

In [4]:
client = boto3.client(
            "s3",
            endpoint_url="http://minio:9000/",
            aws_access_key_id='minio',
            aws_secret_access_key='minio123',
            config=Config(signature_version="s3v4"),
        )

In [8]:
client.list_objects_v2(Bucket='weather')

{'ResponseMetadata': {'RequestId': '18518B32FF841855',
  'HostId': 'dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'accept-ranges': 'bytes',
   'content-length': '480',
   'content-type': 'application/xml',
   'server': 'MinIO',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'vary': 'Origin, Accept-Encoding',
   'x-amz-id-2': 'dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8',
   'x-amz-request-id': '18518B32FF841855',
   'x-content-type-options': 'nosniff',
   'x-ratelimit-limit': '16853',
   'x-ratelimit-remaining': '16853',
   'x-xss-protection': '1; mode=block',
   'date': 'Sat, 12 Jul 2025 15:39:41 GMT'},
  'RetryAttempts': 0},
 'IsTruncated': False,
 'Contents': [{'Key': 'weather_data.json',
   'LastModified': datetime.datetime(2025, 7, 12, 15, 28, 17, 742000, tzinfo=tzlocal()),
   'ETag': '"bc819e80f38df8a6c1c96cf158632caf"',
   'Size': 597,
   'StorageClass': 'STANDARD'}],


In [1]:
import logging
import os

from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, LongType, DoubleType, StringType

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s")
logger = logging.getLogger("MinIOSparkJob")

spark = SparkSession.builder.config("spark.jars", "/opt/jars/aws-java-sdk-bundle-1.11.901.jar,/opt/jars/hadoop-aws-3.3.1.jar,/opt/jars/postgresql-42.7.6.jar,/opt/jars/clickhouse-jdbc-0.9.0.jar,/opt/jars/hadoop-aws-3.3.1-javadoc.jar").getOrCreate()

def load_config(spark_context: SparkContext):
    spark_context._jsc.hadoopConfiguration().set("fs.s3a.access.key", "minio")
    spark_context._jsc.hadoopConfiguration().set("fs.s3a.secret.key",
                                                 "minio123")
    spark_context._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "http://minio:9000/")
    spark_context._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false")
    spark_context._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
    spark_context._jsc.hadoopConfiguration().set("fs.s3a.attempts.maximum", "1")
    spark_context._jsc.hadoopConfiguration().set("fs.s3a.connection.establish.timeout", "5000")
    spark_context._jsc.hadoopConfiguration().set("fs.s3a.connection.timeout", "10000")

In [8]:
from pyspark.sql import functions as F


In [3]:
load_config(spark.sparkContext)

# Define schema for NYC Taxi Data


# Read CSV file from MinIO
df = spark.read.option("header", "true").csv("s3a://new/data.csv")
# Filter dataframe based on passenger_count greater than 6

total_rows_count = df.count()

logger.info(f"Total Rows for NYC Taxi Data: {total_rows_count}")

2025-07-12 16:17:24,527 - MinIOSparkJob - INFO - Total Rows for NYC Taxi Data: 36


In [4]:
df.show()

+----------+----------+------+------+---+
|       Car|     Model|Volume|Weight|CO2|
+----------+----------+------+------+---+
|    Toyoty|      Aygo|  1000|   790| 99|
|Mitsubishi|Space Star|  1200|  1160| 95|
|     Skoda|    Citigo|  1000|   929| 95|
|      Fiat|       500|   900|   865| 90|
|      Mini|    Cooper|  1500|  1140|105|
|        VW|       Up!|  1000|   929|105|
|     Skoda|     Fabia|  1400|  1109| 90|
|  Mercedes|   A-Class|  1500|  1365| 92|
|      Ford|    Fiesta|  1500|  1112| 98|
|      Audi|        A1|  1600|  1150| 99|
|   Hyundai|       I20|  1100|   980| 99|
|    Suzuki|     Swift|  1300|   990|101|
|      Ford|    Fiesta|  1000|  1112| 99|
|     Honda|     Civic|  1600|  1252| 94|
|    Hundai|       I30|  1600|  1326| 97|
|      Opel|     Astra|  1600|  1330| 97|
|       BMW|         1|  1600|  1365| 99|
|     Mazda|         3|  2200|  1280|104|
|     Skoda|     Rapid|  1600|  1119|104|
|      Ford|     Focus|  2000|  1328|105|
+----------+----------+------+----

In [9]:
path = "s3a://weather/weather_data.json"
schema = StructType([
    StructField("latitude", DoubleType()),
    StructField("longitude", DoubleType()),
    StructField("timezone", StringType()),
    StructField("current_weather_units", StructType([
        StructField("time", StringType())
    ])),
    StructField("current_weather", StructType([
        StructField("time", StringType()),
        StructField("temperature", DoubleType()),
        StructField("windspeed", DoubleType())
]))
])
raw_df = spark.read.json(path, schema=schema, multiLine=True)
#raw_df.show(truncate=False)
#raw_df.printSchema()
flat_df = raw_df.select(
    "latitude", "longitude", "timezone",
    F.col("current_weather_units.time").alias("format_time"),
    F.col("current_weather.time").alias("time"),
    F.col("current_weather.temperature").alias("temperature"),
    F.col("current_weather.windspeed").alias("windspeed")
    
)
flat_df.show()


+--------+---------+--------+-----------+----------------+-----------+---------+
|latitude|longitude|timezone|format_time|            time|temperature|windspeed|
+--------+---------+--------+-----------+----------------+-----------+---------+
|   55.75|   37.625|     GMT|    iso8601|2025-07-09T00:15|       21.0|      1.8|
+--------+---------+--------+-----------+----------------+-----------+---------+

